In [54]:
import pandas as pd
from pandas_datareader.data import DataReader
from datetime import date
import matplotlib.pyplot as plt 
from matplotlib import style
import numpy as np
import datetime as dt
from datetime import datetime
from pandas_datareader import data as wb
import quandl
style.use('dark_background') #enable if your Jupyter notebooks are in dark mode

In [80]:
today = dt.date.today()
start_date_input = today - dt.timedelta(days=365*5) 

def import_stock_data(tickers, start = start_date_input, end = today):
    data = pd.DataFrame()
    if len([tickers]) ==1:
        data[tickers] = wb.DataReader(tickers, data_source='yahoo', start = start)['Adj Close']
        data = pd.DataFrame(data)
    else:
        for t in tickers:
            data[t] = wb.DataReader(t, data_source='yahoo', start = start)['Adj Close']
    return(data)


sp500 = import_stock_data(tickers = 'SPY')
msft = import_stock_data(tickers = 'MSFT')
ytm = import_stock_data(tickers = '^TNX')

In [81]:
#Finding the log return on S&P 500
sp500_log_return = np.log(sp500/sp500.shift(1)).dropna()
sp500_log_return.to_numpy()

#Finding the log return on MSFT
msft_log_return = np.log(msft/msft.shift(1)).dropna()
msft_log_return.count()

#Finding the monthly risk free rate
yearly_yield = ytm.to_numpy()
ytm['Monthly Yield'] = yearly_yield / 100 / 12 

In [82]:
msft_log_return

,MSFT
Date,
2016-12-23,-0.004890
2016-12-27,0.000633
2016-12-28,-0.004594
2016-12-29,-0.001430
2016-12-30,-0.012156
...,...
2021-12-14,-0.033130
2021-12-15,0.019036
2021-12-16,-0.029568


In [83]:
ytm

,^TNX,Monthly Yield
Date,,
2016-12-22,2.553,0.002127
2016-12-23,2.543,0.002119
2016-12-27,2.563,0.002136
2016-12-28,2.506,0.002088
2016-12-29,2.477,0.002064
...,...,...
2021-12-14,1.438,0.001198
2021-12-15,1.463,0.001219
2021-12-16,1.422,0.001185


In [70]:
msft_return_extra = msft_log_return - ytm['Monthly Yield'].to_numpy()

ValueError: Unable to coerce to Series, length must be 1: given 1257

In [63]:
msft_log_return
ytm['Monthly Yield']

Date
2016-12-22    0.000021
2016-12-23    0.000021
2016-12-27    0.000021
2016-12-28    0.000021
2016-12-29    0.000021
                ...   
2021-12-14    0.000012
2021-12-15    0.000012
2021-12-16    0.000012
2021-12-17    0.000012
2021-12-20    0.000012
Name: Monthly Yield, Length: 1250, dtype: float64

In [44]:
#Plotting Linear Regression
beta_msft = np.polyfit(sp500_log_return,msft_return_extra,1)
ang_coeff = beta_msft[0]
intercept = beta_msft[1]
beta_eq = ang_coeff*msft_return_extra + intercept 
beta_msft = plt.figure()
ax = beta_msft.subplots()
ax.plot(msft_return_extra, beta_eq,color = 'r', alpha = 0.5, label = 'Regression Line')
ax.scatter(msft_return_extra,sp500_log_return,s = 5, color = 'b', label = 'Data points') #Original data points
ax.set_title('Regression of Microsoft against SP500')
ax.legend()
plt.show()

TypeError: expected 1D vector for x

In [ ]:
#Finding beta, adjusted beta, and the cost of equity
beta = np.polyfit(sp500_log_return,msft_return_extra,1)
my_beta = beta[0]
adj_beta = my_beta * (2/3) + 0.33
mrp = 0.05
rf = 0.035
r = 0.035 + adj_beta * 0.05
print('Microsoft cost of equity is ' + str(r))
print('Microsoft beta found from regression is ' + str(beta[0]))
print('Microsoft adjusted beta is ' + str(adj_beta))

Microsoft cost of equity is 0.07828808504927591
Microsoft beta found from regression is 0.8036425514782771
Microsoft adjusted beta is 0.8657617009855181


In [ ]:
#Inputting the given information
payout_ratio = 0.3455
growth_rate = 0.1497
stock_price = 219.62 
forward_dividend = 2.24 
return_on_new_invstmt = 0.095 
long_term_g = (1-payout_ratio) * return_on_new_invstmt


In [ ]:
#Finding the future value of the dividends
dividend_years = ['2020', '2021', '2022', '2023', '2024', '2025']
dividends = pd.Series(index = dividend_years, dtype = 'float')
dividends['2020'] = 2.24
for dividend_year in range(1,6):
    dividends[dividend_year] = dividends[dividend_year - 1] * (1 + growth_rate)
dividends

2020    2.240000
2021    2.575328
2022    2.960855
2023    3.404095
2024    3.913687
2025    4.499567
dtype: float64

In [ ]:
#Calculating the terminal value in period 4 and period 0 
tv_p_4 = ((dividends[-1] * (1 + long_term_g)) / 
                 (r - long_term_g))
tv_p_0 = tv_p_4/((1+r)**4)
print('Terminal value in period 4 is ' + str(tv_p_4))
print('Terminal value in period 0 is ' + str(tv_p_0))

Terminal value in period 4 is 296.65827069509197
Terminal value in period 0 is 219.44072802429622


In [ ]:
#Finding the price of the shares of MSFT
discount_factors = [(1 / (1 + r)) ** i for i in range (1,6)]
pv_div = (sum(dividends[1:] * discount_factors) +
            tv_p_4 * discount_factors[-1])
print('According to the DDM approach, the price per share of Microsoft is ' + str(pv_div))


According to the DDM approach, the price per share of Microsoft is 217.140188619586


In [ ]:
if pv_div > 212.65 :
    print('Microsoft is undervalued, so it is a buy')
else :
    print('Microsoft is overvalued, so hold')

Microsoft is undervalued, so it is a buy


IndexError: list assignment index out of range

[1.0, 1.05, 1.1025, 1.1576250000000001, 1.2155062500000002]
